# Fitossociologia

> A seguir será apresentada a documentação e exemplo de uso da classe `Fitossociologia`

In [ ]:
#| default_exp Fitossociologia

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
# | export
class Fitossociologia:

    "Cálculo fitossociológico e análise de diversidade"

    def __processamento(
        dados,
        dap: str,
        area_amostrada: float,
        identificacao_arvore: str,
        parcela: str = None,
    ):

        dados = dados.copy()

        dados.rename(columns={identificacao_arvore: "Nome"}, inplace=True)

        dados["AB"] = (np.pi * (dados[dap] * dados[dap])) / 40000

        if parcela is None:

            df_fito = (
                dados.groupby("Nome")
                .agg(N=("Nome", "count"), AB=("AB", "sum"))
                .reset_index()
            )

        else:

            df_fito = (
                dados.groupby("Nome")
                .agg(N=("Nome", "count"), AB=("AB", "sum"), FP=(parcela, "nunique"))
                .reset_index()
            )

            # calculando frequências
            n_parcelas = dados[parcela].nunique()

            df_fito["FA"] = (df_fito["FP"] / n_parcelas) * 100
            df_fito["FR"] = (df_fito["FA"] / (np.sum(df_fito["FA"]))) * 100

        # calculando densidade
        df_fito["DA"] = df_fito["N"] / area_amostrada
        df_fito["DR"] = (df_fito["DA"] / np.sum(df_fito["DA"])) * 100
        df_fito["DoA"] = df_fito["AB"] / area_amostrada
        df_fito["DoR"] = (df_fito["DoA"] / np.sum(df_fito["DoA"])) * 100
        df_fito["IVC"] = (df_fito["DR"] + df_fito["DoR"]) / 2

        return df_fito.sort_values("Nome")

    def __diversidade(df_fito):

        # calculando indíces de diversidade
        S = round(len(df_fito["Nome"]), 0)
        Nid = round(np.sum(df_fito["N"]), 0)
        DA = round((np.sum(df_fito["DA"])), 4)
        Hmax = round((np.log(S)), 4)
        H = round(
            (
                (
                    (Nid * (np.log(Nid)))
                    - (np.sum((df_fito["N"] * np.log(df_fito["N"]))))
                )
                / Nid
            ),
            4,
        )
        Simpson = round(1 - np.sum((df_fito["N"] / Nid) ** 2), 4)
        J = round((H / Hmax), 4)
        diversidade = [S, Nid, DA, Hmax, H, Simpson, J]
        diversidade

        # criando lista com nome das linhas
        linhas = [
            "Número de espécies amostrados – S",
            "Número de indivíduos amostrados – N",
            "Densidade Absoluta – DA",
            "Máxima Diversidade Possível – Ln(S)",
            "Índice de Shannon-Wiener– H",
            "Índice de dominância de Simpson (C)",
            "Índice de Equabilidade de Pielou – J",
        ]

        return pd.DataFrame({"Parâmetro": linhas, "Valor": diversidade})

    def __init__(
        self,
        dados,  # tabela com as informações das árvores
        dap: str,  # nome da coluna com o diamêtro a altura do peito em cm
        area_amostrada: float,  # área amostrada em hectares
        identificacao_arvore: str,  # nome da coluna com nome da árvore
        parcela: str = None,  # nome da coluna com identificação das parcelas
    ):

        tabela_fito = Fitossociologia._Fitossociologia__processamento(
            dados, dap, area_amostrada, identificacao_arvore, parcela
        )

        self.tabela_fitossociologia = tabela_fito
        self.tabela_diversidade = Fitossociologia._Fitossociologia__diversidade(
            tabela_fito
        )

    def salvar_resultados(self):

        self.tabela_fitossociologia.to_csv(
            "tabela_fitossociologia.csv", encoding="utf-8-sig", index=False, sep=";"
        )
        self.tabela_diversidade.to_csv(
            "tabela_diversidade.csv", encoding="utf-8-sig", index=False, sep=";"
        )

        print("Arquivos salvos com sucesso.")

In [ ]:
#| export
#from ForestScience.Fitossociologia import *
import pandas as pd
import numpy as np

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

**Atributos**

**tabela_fitossociologia**: retorna uma tabela (dataframe pandas) com os dados fitossociológicos e as seguintes informações:

| Coluna | Significado                             | Descrição                                                                                 |
|--------------------|---------------------------------------------|-----------------------------------------------------------------------------------------------|
| Nome          | Nome da Espécie                            | Identificação da espécie analisada.                                                          |
| N             | Número de Indivíduos                       | Total de indivíduos de uma espécie.                                                          |
| AB            | Área Basal                                 | Soma das áreas seccionais dos troncos dos indivíduos de uma espécie.                         |
| FP            | Frequência por Parcela                     | Número de parcelas em que uma espécie está presente.                                         |
| FA            | Frequência Absoluta                        | Porcentagem de parcelas em que uma espécie ocorre em relação ao total de parcelas amostradas.|
| FR            | Frequência Relativa                        | Porcentagem da frequência de uma espécie em relação à soma das frequências de todas as espécies.|
| DA            | Densidade Absoluta                         | Número de indivíduos de uma espécie por unidade de área.                                     |
| DR            | Densidade Relativa                         | Proporção da densidade de uma espécie em relação à densidade total de todas as espécies.     |
| DoA           | Dominância Absoluta                        | Área basal de uma espécie por unidade de área.                                               |
| DoR           | Dominância Relativa                        | Proporção da dominância de uma espécie em relação à dominância total de todas as espécies.   |
| IVC           | Índice de Valor de Cobertura               | Média da densidade relativa e dominância relativa, indicando a importância de uma espécie no ecossistema. |

**tabela_diversidade**: retorna uma tabela (dataframe pandas) com os dados de diversidade e as seguintes informações:

| Campo                                   | Significado                                                                                 |
|-------------------------------------------------|-------------------------------------------------------------------------------------------------|
| Número de espécies amostrados – S          | Número total de espécies diferentes identificadas na amostragem.                               |
| Número de indivíduos amostrados – N        | Número total de indivíduos de todas as espécies na amostragem.                                 |
| Densidade Absoluta – DA                    | Soma das densidades absolutas de todas as espécies na área amostrada.                          |
| Máxima Diversidade Possível – Ln(S)        | Diversidade máxima possível, calculada como o logaritmo natural do número de espécies (S).     |
| Índice de Shannon-Wiener – H               | Índice que mede a diversidade considerando o número de espécies e a abundância relativa.       |
| Índice de dominância de Simpson (C)        | Índice que mede a probabilidade de dois indivíduos selecionados ao acaso pertencerem à mesma espécie. |
| Índice de Equabilidade de Pielou – J       | Índice que mede a uniformidade da distribuição dos indivíduos entre as espécies.               |

**Metódos**

**salvar_resultados**(): salva as tabelas em csv.

### Exemplo de uso

<br>
Importe os pacotes necessários: <br>
```sh
from ForestScience.Fitossociologia import *
import pandas as pd
```


In [ ]:
# leitura dos dados
dados = pd.read_csv('tabela_teste_fitossociologia.csv')
dados.head()

,parcela,especie,dap
0,5,Stenocalyx dysentericus,10.50
1,3,Dimorphandra mollis,13.37
2,6,Stenocalyx dysentericus,16.87
3,5,Aspidosperma subincanum Mart,13.05
4,2,Schinus terebinthifolia,15.92


In [ ]:
# Inicializa a classe Fitossociologia para realizar cálculos fitossociológicos e de diversidade
fito = Fitossociologia(
    dados=dados,
    dap="dap",
    area_amostrada=0.5,
    identificacao_arvore="especie",
    parcela="parcela",
)

In [ ]:
# exibindo a tabela com os dados fitossociologicos
fito.tabela_fitossociologia

,Nome,N,AB,FP,FA,FR,DA,DR,DoA,DoR,IVC
0,Alibertia edulis,1,0.019113,1,20.0,2.631579,2.0,0.751880,0.038227,0.753324,0.752602
1,Aspidosperma subincanum Mart,1,0.013376,1,20.0,2.631579,2.0,0.751880,0.026751,0.527174,0.639527
2,Astronium fraxinifolium,12,0.238957,4,80.0,10.526316,24.0,9.022556,0.477915,9.418095,9.220326
3,Brosimum gaudichaudii,3,0.088937,2,40.0,5.263158,6.0,2.255639,0.177874,3.505305,2.880472
4,Byrsonima coccolobifolia,2,0.028860,1,20.0,2.631579,4.0,1.503759,0.057720,1.137476,1.320618
5,Caryocar brasiliensis,6,0.080987,4,80.0,10.526316,12.0,4.511278,0.161974,3.191962,3.851620
6,Curatella americana Lineus,1,0.032589,1,20.0,2.631579,2.0,0.751880,0.065178,1.284442,1.018161
7,Dalbergia miscolobium,2,0.053647,2,40.0,5.263158,4.0,1.503759,0.107294,2.114396,1.809078
8,Dimorphandra mollis,6,0.195633,3,60.0,7.894737,12.0,4.511278,0.391265,7.710527,6.110903
9,Kielmeyera coriacea Mart. & Zucc.,1,0.006691,1,20.0,2.631579,2.0,0.751880,0.013382,0.263716,0.507798


In [ ]:
# exibindo a tabela com os dados de diversidade
fito.tabela_diversidade

,Parâmetro,Valor
0,Número de espécies amostrados – S,17.0000
1,Número de indivíduos amostrados – N,133.0000
2,Densidade Absoluta – DA,266.0000
3,Máxima Diversidade Possível – Ln(S),2.8332
4,Índice de Shannon-Wiener– H,1.9800
5,Índice de dominância de Simpson (C),0.7598
6,Índice de Equabilidade de Pielou – J,0.6989


In [ ]:
# salvando os resultados do processamento
fito.salvar_resultados()

Arquivos salvos com sucesso.
